In [8]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [3]:
df = pd.read_csv('RecipeNLG/RecipeNLG_dataset.csv')
print(f'df size before: {len(df)}')
df.head()

df size before: 2231142


,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
df = df[~df['link'].str.contains('www.cookbooks.com')]
print(f'df size without cookbooks.com: {len(df)}')
df = df[~df['link'].str.contains('www.allrecipes.com')]
print(f'df size without allrecipes.com: {len(df)}')
df.head()

In [15]:
def stringify_recipe(recipe):
    title = recipe['title']
    ingredients = eval(recipe['ingredients'])
    directions = eval(recipe['directions'])
    ner = eval(recipe['NER'])

    return json.dumps({
        'ner': ner,
        'title': title,
        'ingredients': ingredients,
        'directions': directions,
    })

jsonified_recipes = df.apply(stringify_recipe, axis=1)

In [16]:
for r in jsonified_recipes[:2]:
    print(json.dumps(json.loads(r), indent=3))
    print()

{
   "ner": [
      "brown sugar",
      "milk",
      "vanilla",
      "nuts",
      "butter",
      "bite size shredded rice biscuits"
   ],
   "title": "No-Bake Nut Cookies",
   "ingredients": [
      "1 c. firmly packed brown sugar",
      "1/2 c. evaporated milk",
      "1/2 tsp. vanilla",
      "1/2 c. broken nuts (pecans)",
      "2 Tbsp. butter or margarine",
      "3 1/2 c. bite size shredded rice biscuits"
   ],
   "directions": [
      "In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.",
      "Stir over medium heat until mixture bubbles all over top.",
      "Boil and stir 5 minutes more. Take off heat.",
      "Stir in vanilla and cereal; mix well.",
      "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.",
      "Let stand until firm, about 30 minutes."
   ]
}

{
   "ner": [
      "beef",
      "chicken breasts",
      "cream of mushroom soup",
      "sour cream"
   ],
   "title": "Jewell Ball'S Chicken",
   "ingr

In [ ]:
chars = sorted(list(set(all_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

seq_length = 100
dataX = []
dataY = []

In [ ]:
for i in range(0, len(all_text) - seq_length, 1):
    seq_in = all_text[i:i + seq_length]
    seq_out = all_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
X = np.reshape(dataX, (n_patterns, seq_length, 1)) / float(len(chars))
y = to_categorical(dataY)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()
model.save('neural_nourishment.keras')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train, epochs=15, batch_size=128, validation_data=(X_test, y_test))